In [1]:
import numpy as np
import os

###1. feat center
fp1=open('test_emo.txt','r')
fp2=open('train_emo.txt','r')
l1=fp1.readlines()
l2=fp2.readlines()


feat_center=np.ndarray((7,2048))
feat_buff_list=[]
for i in range(7):feat_buff_list.append([])#get empty buff for saving all feat
for i,l in enumerate(l1+l2):
	lname,lb=l[:-1].split(' ')

	nparr=np.load('npy_imgfeat/'+lname.replace('.jpg','.npy'))
	nparr=nparr[0,...]
	# if i<10:print nparr[1000:1050]
	feat_buff_list[int(lb)]+=[nparr]

for i in range(7):
	print len(feat_buff_list[i])
	feat_center[i,:]=np.median(np.array(feat_buff_list[i]),axis=0)
	# print feat_center[i,1000:1050]
 
###2. disntance to own center
dist2center=[]
for i in range(7):
	dist2center.append([])
for i,l in enumerate(l1+l2):
	lname,lb=l[:-1].split(' ')
	nparr=np.load('npy_imgfeat/'+lname.replace('.jpg','.npy'))
	lb=int(lb)
	dist=np.sum(np.square(nparr-feat_center[lb,:]))/1024
	dist2center[lb]+=[[lname,dist]]	

for i in range(7):
	dist2center[i]=sorted(dist2center[i],key=lambda x: x[1])
	# dist2center[i,:]=sorted(dist2center[i,:],key=operator.itemgetter(1))
	# print dist2center[i][:10]

1905
975
1381
3636
2381
2485
1994


In [2]:
###3. get most close 90%

#generate new files 
fn1=open('new_train_emo.txt','w')
fn2=open('new_test_emo.txt','w')

fw_line_tr=[]
fw_line_ts=[]
for i in range(7):
	print 'emo: '+str(i)
	num=len(dist2center[i])
	print dist2center[i][0][1]
	for j in range(int(num*0.9)):
		if 'webemo_tr' in dist2center[i][j][0]:
			fname=dist2center[i][j][0]
			fw_line_tr+=[fname+' '+str(i)]
		else:
			fname=dist2center[i][j][0]
			fw_line_ts+=[fname+' '+str(i)]
	print dist2center[i][j][1]
import random

random.shuffle(fw_line_ts)
random.shuffle(fw_line_tr)

for f in fw_line_tr:
	fn1.write(f+'\n')
for f in fw_line_ts:
	fn2.write(f+'\n')

#checkout confusion matrix
confusion_mt=np.ndarray((7,7))
for i,l in enumerate(l1+l2):
	lname,lb=l[:-1].split(' ')
	nparr=np.load('npy_imgfeat/'+lname.replace('.jpg','.npy'))
	lb=int(lb)
	for j in range(7):
		dist=np.sum(np.square(nparr-feat_center[j,:]))/2048
		confusion_mt[lb,j]+=dist
for i in range(7):
	confusion_mt[i,:]/=len(feat_center[i])

print confusion_mt

emo: 0
9.49456747272
45.7427367192
emo: 1
8.26589280402
42.1484923972
emo: 2
8.21139572747
46.9413262017
emo: 3
9.38532752806
42.0609984155
emo: 4
7.20854090149
42.8680814231
emo: 5
9.74687889948
44.0493429873
emo: 6
10.1551246729
47.182200975
[[ 14.1651113   15.52895355  17.00977519  16.5072325   16.57259936
   15.48842945  18.03372207]
 [  7.40285905   6.67683357   8.14907773   7.92842734   6.9680425
    7.47033418   8.46313662]
 [ 12.00244688  11.94540408  10.33500943  13.07280886  12.2673534
   12.27212878  10.84814928]
 [ 30.50475392  30.70308675  34.42804112  25.34804856  33.44035426
   30.85824849  35.46099066]
 [ 18.80285275  16.401033    19.85686647  20.57292181  15.97260416
   19.11902384  20.03286767]
 [ 18.93833028  19.19920141  21.4626177   20.53114385  20.71361085
   17.28771786  23.42928063]
 [ 18.32763146  17.80860841  15.68496615  19.27817958  17.81276594
   19.20800411  14.78308224]]


In [3]:
fn1=open('new_train_emo.txt','r')
fn2=open('new_test_emo.txt','r')
ln1=fn1.readlines()
ln2=fn2.readlines()

###5 try out the new confusion matrix

confusion_mt=np.ndarray((7,7))
for i,l in enumerate(ln1+ln2):
	lname,lb=l[:-1].split(' ')
	nparr=np.load('npy_imgfeat/'+lname.replace('.jpg','.npy'))
	lb=int(lb)
	for j in range(7):
		dist=np.sum(np.square(nparr-feat_center[j,:]))/2048
		confusion_mt[lb,j]+=dist
for i in range(7):
	confusion_mt[i,:]/=len(feat_center[i])*0.9

print confusion_mt

[[ 12.75858424  14.15557856  15.37637761  15.19890705  15.13475988
   13.99992953  16.43976358]
 [  6.64372261   5.96847399   7.27155654   7.24854576   6.22326034
    6.68602654   7.61147747]
 [ 10.99313489  10.96044259   9.25579068  12.13448529  11.24862903
   11.21566711   9.79076307]
 [ 27.82710198  28.03863405  31.38434209  23.02815136  30.59785693
   28.11898594  32.44548081]
 [ 16.8297464   14.5126955   17.59110098  18.79824696  13.98885662
   17.05900559  17.83146365]
 [ 16.93996993  17.1742661   19.07280975  18.72981004  18.51013032
   15.29941919  21.01552023]
 [ 16.7825751   16.32661158  13.9211581   18.01705447  16.2528047
   17.5957985   13.0960865 ]]
